# Front End Lab 1: Amplification

### EE 16B: Designing Information Devices and Systems II, Fall 2015

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-


##Table of Contents

* [Introduction](#intro)
* [Task 0: Test Signal Setup](#task0)
* [Task 1: Instrumentation Amplifier](#task1)
* [Task 2: Non-inverting Amplifiers](#task2)

<a id='intro'></a>
## Introduction
Welcome to the third module - Front End! In the first module we learned how the brain wave data we record can be used to predict the behavior of the subject while in the second module we learned how we can convert an analog signal (something from the natural world) to a digital signal (something representable in a computer). Recall the block diagram below from the DSP module:

<img style="width: 500px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/adc-dac.png">

Our goal in this module is to build the <b>Signal Conditioning</b> block in the top row above. Although the voltages we get from the brain are analog signals, they are very <i>very</i> small, usually in the order of ~1$\mu$V. We need to provide some amplification and filtering such that our signal is within the acceptable range for our ADC input (in our case, the Launchpad). The block diagram of the circuit we will implement in this module is shown below.

<img style="width: 750px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-block.png">

We will see how each block above is needed by trying to use the Launchpad ADC at every stage. We will also try using both sinusoids and a simulated Local Field Potential (LFP) BMI signal from the first BMI lab.

This week we will build a set of amplifiers that we will slowly build up from as we encounter different issues. <b>As you can see above, the circuit we will be building is quite complicated. Please try to be very neat and plan ahead in your breadboard layout.</b>

###<span style="color:green">IMPORTANT: KEEP THE CIRCUIT IN THIS LAB FOR NEXT LAB</span>
Otherwise you will have to rebuild everything again next week.

###Quickly skim through <a href="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab/circuit-debugging-tips.pdf">this document</a> on circuit debugging before you begin.

<a id='task0'></a>
##<span style="color:blue">Task 0: Test Signal Setup</span>

###Materials
- Function generator + cable
- 1 1M$\Omega$ resistor
- 1 100$\Omega$ resistor
- Breadboard
- Oscilloscope + 2 probes
- Launchpad + USB cable

To test our signal conditioning, we will provide some BMI waveform in a loop through the function generator. Because a real BMI signal is around 1$\mu$V while the function generator produces signals around 100mV, we will divide down the generator test signal with a resistive divider. Build a ~1:10000 resistive divider in one end of your breadboard. <b>DO NOT CONNECT ANY NODE TO THE GROUND RAIL.</b>

**<span style="color:red">If we supply a signal with 100mV amplitude, what is the output amplitude?</span>**

YOUR ANSWER HERE

**<span style="color:red">Test your resistive divider by inputting 25V in the input. Use a digital multimeter as a voltmeter to measure the output voltage - make sure it is around 2.5mV.</span>**

We will start with a signal 10 times larger than what it is supposed to be since it is easier to debug and see the signal without too much amplification. Thus, we will use a 1Vpp (volt peak-to-peak) signal from the function generator. Next week we will reduce this to 0.1Vpp.

The BMI waveform has been uploaded to the function generator as `notMoving_B.arb`. To load this arbitrary waveform, press the <b>Waveforms</b> button, select <b>Arb</b>, then <b>Arbs</b>, then <b>Select Arb</b>, and find the file <b>`notMoving_B.arb`</b>. Then, press the <b>Parameters</b> button and change the parameters to:
- Sampling rate: <b>1 kSa/s</b>
- Amplitude: <b>1 Vpp</b>
- Offset: <b>0 V</b>

Lastly, turn the channel on by pressing the button above your cable (the channel number) and then the blue-gray button next to the power button (labelled Output on the screen). Use the oscilloscope to make sure the function generator is producing the correct BMI signal at around 1.7V peak-to-peak. This happens because the function generator expects a 50$\Omega$ load while the load in the circuit is much more than 50$\Omega$, but we do not need to worry about it. Try probing the output of the resistive divider - the signal is so small you can't even see it in an oscilloscope!

Let's try to use the Launchpad ADC to read the signal anyway. Upload the sketch <b>`adc_read.ino`</b> to your Launchpad - make sure the board is set to MSP430F5529 (25MHz) in Energia. Open Device Manager to find the COM port of the Application UART. Hit the reset button on the Launchpad. Connect the ground of your launchpad to the negative terminal of the resistive divider and pin `P6.0` to the output of the resistive divider.

**<span style="color:blue">HOW MANY THINGS SHOULD BE CONNECTED TO THE GROUND RAILS AT THIS POINT?</span>**
**ZERO!**

 Then open the terminal (run `cmd`) and navigate to the directory containing the lab files and run (replacing `###` with the right port number):

`python adc_read.py -D COM###`

This brings up a plot of the output of your ADC, normalized to [0,1]. The ADC is sampling at 333Hz (specified in the Launchpad sketch). You probably won't see anything at this point since the signal is too small! As you can imagine, we need a lot of amplification. The series of amplifiers we will build today is shown below:

<img style="width: 750px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-block-1.png">

<a id='task1'></a>
##<span style="color:blue">Task 1: Instrumentation Amplifier</span>

###Materials
- Task 0 setup
- 2 <a href="http://www.ti.com/lit/ds/symlink/lmc6482.pdf">LMC6482</a> Dual Op-amp
- 2 100k$\Omega$ resistors
- 3 10k$\Omega$ resistors
- 2 250k$\Omega$ resistors

Because the signal is so small, we will have to amplify it by several orders of magnitude in order to apply the appropriate signal processing. Therefore, we will first send our signal through an instrumentation amplifier. 

<img style="width:450px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-instr-amp.png">

An instrumentation amplifier takes a differential input (2-terminal input) and has a single-ended output. To analyze this circuit, we can break it up to 3 smaller circuits:

<img style="width:450px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-inamp.png">

The ground between the top and bottom blocks is called a "virtual ground" since it is not really a ground, but acts as one because of the differential signal.

**<span style="color:red">Using the blocks above, derive an expression for $V_{\text{out}}$ as a function of $(V_p - V_n)$</span>** (Hint: you've seen the individual blocks - what are their gains?)

YOUR ANSWER HERE

**<span style="color:red">If $R_1 = R_3 = 10k\Omega, R_2 = 100k\Omega, R_4 = 250k\Omega$, what is the overall gain?</span>**

YOUR ANSWER HERE

Now build your instrumentation amplifier using the <a href="http://www.ti.com/lit/ds/symlink/lmc6482.pdf">LMC6482</a> op-amps. Use +5V and -5V for your op-amp supplies. <b>DO NOT FORGET TO SET THE CURRENT LIMIT.</b> The pinouts of the op-amp are below. There are 2 $250k\Omega$ resistors in your packet that does not look like regular resistors - they are brown in color with the writing `2503F`. Connect the output of the resistive divider to the input of the instrumentation amplifier. <b>DO NOT CONNECT ANY OF THE INPUTS TO GROUND.</b> This is because we are treating the signal as differential.

<img style="width:400px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/opamp-pins.png">

Once you're done, use the oscilloscope to make sure the output looks similar to the waveform below. The yellow waveform is the output of the function generator while the green is the output of the instrumentation amplifier.

<img style="width:600px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-scope-instr-amp.png">

**<span style="color:red">Checkoff: Show your GSI the output of the instrumentation amplifier.</span>**

Remember that the test signal right now is 10 times larger than what it's supposed to be! <b>Change the amplitude parameter of the function generator to 100mVpp</b> and probe the output again. We still need a lot of gain!

<a id='task2'></a>
##<span style="color:blue">Task 2: Non-inverting Amplifiers</span>

###Materials
- Task 1 setup
- 1 LMC6482 Dual Op-amp
- resistors

As you can see, we probably do not have enough gain with just the instrumentation amplifier. To solve this issue, build 2 non-inverting amplifiers after the instrumentation amplifier using 1 op-amp chip. Leave some space for a mystery circuit that goes between the instrumentation amplifier and the non-inverting amplifiers. Build:
- a non-inverting amplifier with gain 100
- a non-inverting amplifier with gain ???

Compute the last amplifier gain assuming our input signal after divider is 10$\mu V$ and we want an output swing of 3V.

<b>Test each circuit individually</b>: 
Use the function generator to generate a sinusoid with reasonable amplitude, feed it to the first non-inverting amplifier and see if the output is what you expected. Make sure to probe both the input and the output - the output should be around 100 times greater. Next, do the same thing just for the second non-inverting amplifier.

**<span style="color:red">Checkoff: Show your GSI the input and output waveforms of both of your non-inverting amplifiers.</span>**

Now connect these amplifiers together in a cascade as seen in the block diagram below.

<img style="width: 750px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-block-1.png">

Change your function generator to output the BMI waveform again; remember to use the parameters below:
- Sampling rate: <b>1 kSa/s</b>
- Amplitude: <b>100 mVpp</b>
- Offset: <b>0 V</b>

Probe the output of the last non-inverting amplifier. (Don't freak out if you don't see a nice waveform - or any waveform at all...)

**<span style="color:red">What problem do you see? Why does this happen? How can we potentially solve this issue?</span>** Hint: What happens if your instruentation amplifier has a small DC offset?

YOUR ANSWER HERE

The output waveform right now is railing to -5V. If we were to connect this to our Launchpad ADC, the Launchpad pin would be burnt - **DO NOT DO THIS!!** We need to find a solution for this issue in the next lab.

###<span style="color:green">IMPORTANT: KEEP THE CIRCUIT IN THIS LAB FOR NEXT LAB</span>
Otherwise you will have to rebuild everything again next week.

Now you have a large amplification, but you still can't get the Launchpad ADC to sample the voltage levels nicely. We will continue to refine our signal processing chain next week.